In [7]:
from lib.measurement import MultiQubitMeasurement
from lib.formatting import format_bytes, bitstring_to_filestring
from lib.state_io import save_state_vector

####

from pathlib import Path

import jax.numpy as jnp
from jax import random


rng_seed = 42

data_dir = Path("data")
data_dir.mkdir(parents=True, exist_ok=True)

print(f"Random seed is {rng_seed}")
print(f"Data will be saved to {data_dir.resolve()}")

Random seed is 42
Data will be saved to /Users/Tonni/Desktop/master-code/neural-quantum-tomo/case_studies/shadow_rbm/data


In [8]:
#### PHASE AUGMENTED W STATE

num_qubits = 8

state_dim = 1 << num_qubits                 # bit shifting a ...0001 bitstring is the same as 2**num_qubits

rng_phase = random.PRNGKey(rng_seed)
one_hot_thetas = random.uniform(rng_phase, shape=(num_qubits,), minval=0.0, maxval=2 * jnp.pi)
one_hot_amplitudes = jnp.exp(1j * one_hot_thetas) / jnp.sqrt(num_qubits)

one_hot_indices = 1 << jnp.arange(num_qubits - 1, -1, -1)   # we shift a single bit left with descending ints

w_aug = jnp.zeros(state_dim, dtype=jnp.complex64).at[one_hot_indices].set(one_hot_amplitudes)

print("First 10 amplitudes:")
for idx in range(10):
    print(f"{idx:0{num_qubits}b}: {w_aug[idx]:.8f}")

print(f"\nSize of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

First 10 amplitudes:
00000000: 0.00000000+0.00000000j
00000001: 0.35348165+0.00712282j
00000010: 0.34222963+0.08876304j
00000011: 0.00000000+0.00000000j
00000100: -0.07679566+0.34511220j
00000101: 0.00000000+0.00000000j
00000110: 0.00000000+0.00000000j
00000111: 0.00000000+0.00000000j
00001000: 0.02867402-0.35238871j
00001001: 0.00000000+0.00000000j

Size of state vector in memory: 2.00 KB 



In [9]:
w_amp_file_name = f"w_aug_{num_qubits}.txt"
w_amp_file_path = data_dir / w_amp_file_name
save_state_vector(w_aug, w_amp_file_path)

State vector written to data/w_aug_8.txt (8960 bytes)


In [13]:
num_samples_comp = 10_000
rng_comp = random.PRNGKey(rng_seed + 1)

basis_comp = ['Z'] * num_qubits
basis_comp_str = ''.join(basis_comp)

filename_comp = f"w_aug_{basis_comp_str}_{num_samples_comp}.txt"
filepath_comp = data_dir / filename_comp

with open(filepath_comp, "w") as f:
    measurement = MultiQubitMeasurement(basis_comp)
    samples = measurement.sample_state(w_aug, num_samples_comp, rng=rng_comp)
    for sample in samples:
        filestring = bitstring_to_filestring(sample, basis_comp) + "\n"
        f.write(filestring)

    print(f"Wrote {num_samples_comp} samples to {filename_comp}")

Constructing basis ZZZZZZZZ: 100%|██████████| 256/256 [00:00<00:00, 2639.39it/s]


Wrote 10000 samples to w_aug_8_10000.txt


In [14]:
num_samples_mixed = 30_000
rng_mixed = random.PRNGKey(rng_seed + 2)

filename_mixed = f"w_aug_{num_qubits}MIXED_{num_samples_mixed}.txt"
filepath_mixed = data_dir / filename_mixed


def random_basis_string(rng, num_qubits):
    paulis = ['X', 'Y', 'Z']
    idxs = random.randint(rng, shape=(num_qubits,), minval=0, maxval=3)
    return [paulis[i] for i in idxs.tolist()]

with open(filepath_mixed, 'w') as f:
    for i in range(num_samples_mixed):
        rng_mixed, rng_basis, rng_sample = random.split(rng_mixed, 3)

        measurement_dirs = random_basis_string(rng_basis, num_qubits)
        measurement = MultiQubitMeasurement(measurement_dirs)

        sample = measurement.sample_state(w_aug, 1, rng=rng_sample)[0]
        formatted = bitstring_to_filestring(sample, measurement_dirs)

        f.write(formatted + "\n")

        if i % 1000 == 0:
            print(f"{i} samples written...")

print(f"Done. {num_samples_mixed} samples saved to {filename_mixed}.")

Constructing basis ZYXYZYZX: 100%|██████████| 256/256 [00:00<00:00, 2531.97it/s]


0 samples written...


Constructing basis ZYXXXYZY: 100%|██████████| 256/256 [00:00<00:00, 2575.35it/s]


1000 samples written...


Constructing basis YXZYXYXY: 100%|██████████| 256/256 [00:00<00:00, 2353.05it/s]


...

28000 samples written...


Constructing basis XZYZZXZY: 100%|██████████| 256/256 [00:00<00:00, 2668.05it/s]


29000 samples written...


Constructing basis XYXYYXYX: 100%|██████████| 256/256 [00:00<00:00, 2611.72it/s]

Done. 30000 samples saved to w_aug_mixed8_30000.txt.
